In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that
# running it shouldn't break anything for u
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [17]:
# importing the model config
from config import *

# importing N-GPT
from model import Model

# the config
from config import ModelConfig

# imports for the tokenizer
import pickle
from tokenizer.tokenizer import BPE_Tokenizer

# does the actual inference
from inference import generate

# used to save & load models
import json
from dataclasses import asdict

# Load a Pretrained Model

In [14]:
# pretrained model options:
# 2m parameters, context length = 256, trained for 500 iterations w/ batch size of 32 and no dropout: 'Llama3_2024-04-19|04-00-15'
# 2m parameters, context length = 512, trained for 1000 iterations w/ batch size 32 and dropout 0.1: 'Llama3_2024-04-19|15-18-16'
# 3m parameters, context length = 512, trained for 1300 iterations w/ batch size of 24 and dropout 0.1: 'Llama3_2024-04-19|17-21-51'
model_name = 'Model_2024-11-07|13-54-51'

# Deserialize the JSON file back to a dictionary
with open(f'models/{model_name}/model_config.json', 'r') as f:
    config_dict = json.load(f)

# Convert the dictionary back to a Config object
cfg = ModelConfig(**config_dict)
cfg.device = ('cuda' if torch.cuda.is_available() 
              else 'mps' if torch.backends.mps.is_available() 
              else 'cpu')

with open(f'tokenizer/models/{cfg.vocab_len - 3}.model', 'rb') as f:
        tokenizer_data = pickle.load(f)
tokenizer = BPE_Tokenizer(tokenizer_data['merges']) 

# Initialize a blank model
model = Model(cfg).to(cfg.device) 

# Load the saved state dictionary
path = f'models/{model_name}/model.pth'
model.load_state_dict(torch.load(path, map_location="cpu"))

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

# If you only plan to do inference, switch to evaluation mode
model.eval()

528.544 K parameters


/var/folders/14/tymmlq9x35718s1q9_f9zkp80000gn/T/ipykernel_16409/861789102.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_lo

Model(
  (precompute_freqs): PrecomputeRotaryFrequencies()
  (token_embedder): Embedding(1024, 64)
  (layers): ModuleList(
    (0-7): 8 x Layer(
      (attn): SelfAttention(
        (Wq): Linear(in_features=64, out_features=64, bias=False)
        (Wk): Linear(in_features=64, out_features=64, bias=False)
        (Wv): Linear(in_features=64, out_features=64, bias=False)
        (Wo): Linear(in_features=64, out_features=64, bias=False)
      )
      (mlp): MLP(
        (Wup): Linear(in_features=64, out_features=170, bias=False)
        (Wgate): Linear(in_features=64, out_features=170, bias=False)
        (Wdown): Linear(in_features=170, out_features=64, bias=False)
      )
    )
  )
  (output): Linear(in_features=64, out_features=1024, bias=False)
  (criterion): CrossEntropyLoss()
)

# Inference

In [21]:
output = generate("JULIET:\nO Romeo, Romeo! wherefore art thou", model, tokenizer, temperature=0.7, max_gen_len = 32)
print(output)

JULIET:
O Romeo, Romeo! wherefore art thou now re�ostity oldhingort kn myROMEseful EDWARDotK S g ouringBe_ Tent but marEi in nowir
